In [1]:
!pip install simpletransformers
!pip install gensim
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21

In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
from sklearn.metrics import classification_report
from gensim.utils import simple_preprocess

# Reading Train and Val Datasets for Training the Models

In [3]:
train_data = pd.read_csv('TaskA-TrainingSet.csv')
val_data = pd.read_csv('TaskA-ValidationSet.csv')

unq_headers = list(train_data['section_header'].unique())
label2id = {i:idx for idx,i in enumerate(unq_headers)}
id2label = {idx:i for idx,i in enumerate(unq_headers)}

In [4]:
train_data['section_header'] = train_data['section_header'].replace(label2id)
val_data['section_header'] = val_data['section_header'].replace(label2id)

train_data['text'] = train_data['dialogue'].map(lambda x: x.replace('\n',' ').replace('\r',' '))
train_data['labels'] = train_data['section_header']

val_data['text'] = val_data['dialogue'].map(lambda x: x.replace('\n',' ').replace('\r',' '))
val_data['labels'] = val_data['section_header']

# Calculating Class weights because of Class imbalance in the dataset

In [5]:
from sklearn.utils import class_weight
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(train_data['labels'].tolist()),y=train_data['labels'])

In [6]:
train_df = pd.DataFrame(train_data)
eval_df = pd.DataFrame(val_data)

model1_args = ClassificationArgs(num_train_epochs=10)
model1_args.overwrite_output_dir = True

model1 = ClassificationModel(
    "bert", 
    "emilyalsentzer/Bio_ClinicalBERT",
    num_labels=20,
    weight=list(class_weights),
    args=model1_args,
    use_cuda=True
) 
model1.train_model(train_df)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

  0%|          | 0/1201 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/151 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/151 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/151 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/151 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/151 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/151 [00:00<?, ?it/s]

(1510, 0.8818239045844395)

In [7]:
predictions = model1.predict(val_data['text'].tolist())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [9]:
print(classification_report(val_data['section_header'].tolist(), predictions[0]))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90        20
           1       0.86      0.86      0.86         7
           2       0.43      0.75      0.55         4
           3       0.80      1.00      0.89         4
           4       1.00      0.50      0.67         4
           5       0.85      1.00      0.92        22
           6       1.00      1.00      1.00         8
           7       0.00      0.00      0.00         1
           8       0.40      0.50      0.44         4
           9       0.89      0.73      0.80        11
          10       0.50      1.00      0.67         2
          11       0.50      1.00      0.67         1
          12       1.00      0.67      0.80         3
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         3
          15       1.00      1.00      1.00         1
          16       0.00      0.00      0.00         1
          17       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
strong_indicator_words = {
    'GYNHX': ['pregnant', 'miscarriage', 'abortion', 'periods', 'menstrual', 'obstetrician', 'gyneco'],
    'IMMUNIZATIONS': ['vaccine', 'immunization']
}

new_predictions=[]
for i in val_data['text'].tolist():
  if any(word in i for word in strong_indicator_words['GYNHX']):
    new_predictions.append(19)
  elif any(word in i for word in strong_indicator_words['IMMUNIZATIONS']):
    new_predictions.append(15)
  else:
    new_predictions.append(model1.predict([i])[0][0])

print(classification_report(val_data['section_header'].tolist(), new_predictions))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.90      0.95      0.93        20
           1       0.86      0.86      0.86         7
           2       0.43      0.75      0.55         4
           3       0.80      1.00      0.89         4
           4       1.00      0.50      0.67         4
           5       0.85      1.00      0.92        22
           6       1.00      1.00      1.00         8
           7       0.00      0.00      0.00         1
           8       0.40      0.50      0.44         4
           9       0.89      0.73      0.80        11
          10       0.50      1.00      0.67         2
          11       0.50      1.00      0.67         1
          12       1.00      0.67      0.80         3
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         3
          15       1.00      1.00      1.00         1
          16       0.00      0.00      0.00         1
          17       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
test_predictions_with_ids = []
for i,j in zip(val_data['ID'].tolist(), new_predictions):
  test_predictions_with_ids.append([i,id2label[j]])

test_df = pd.DataFrame(test_predictions_with_ids, columns = ['ID', 'SystemOutput1'])
test_df.to_csv('final_taskA_classification_output.csv', index=False)